In [1]:
import openml
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from automl import AutoML

## Функция для запуска пайплайна



In [2]:
def run_automl_pipeline(
    task_id: int,
    test_size: float = 0.2,
    random_state: int = 42,
    n_jobs: int = 16,
    tuning_timeout: int = 10,
    verbose: bool = True
) -> pd.DataFrame:
    """
    Запускает полный пайплайн AutoML для задачи из OpenML со стратифицированным разделением на train/test.
    
    Параметры:
    ----------
    task_id : int
        ID задачи из OpenML
    test_size : float, default=0.2
        Доля данных для тестового набора
    random_state : int, default=42
        Random state для воспроизводимости
    n_jobs : int, default=16
        Количество параллельных процессов
    tuning_timeout : int, default=10
        Таймаут для тюнинга модели (в секундах)
    verbose : bool, default=True
        Выводить ли информационные сообщения
    
    Возвращает:
    -----------
    pd.DataFrame
        Датафрейм с предсказаниями и метриками:
        - predictions_train: предсказания на обучающем наборе
        - predictions_test: предсказания на тестовом наборе
        - y_true: истинные значения для всех данных
        - y_true_train: истинные значения для train
        - y_true_test: истинные значения для test
        - is_test: флаг, является ли строка тестовой
        - oof_score: OOF score на train
        - train_score: ROC-AUC на train
        - test_score: ROC-AUC на test
        - task_id: ID задачи
        - dataset_name: название датасета
    """
    import traceback
    
    results_dict = {}
    
    try:
        if verbose:
            print(f"[INFO] Начало загрузки данных из OpenML (task_id={task_id})...")
        
        # Загрузка задачи и датасета
        task = openml.tasks.get_task(task_id)
        dataset = task.get_dataset()
        
        if verbose:
            print(f"[INFO] Задача загружена: {task.task_id}")
            print(f"[INFO] Датасет загружен: {dataset.name}")
        
        # Получение параметров
        params = task.estimation_parameters
        n_folds = int(params.get("number_folds", 3)) - 1
        class_labels = task.class_labels
        
        if verbose:
            print(f"[INFO] Количество фолдов: {n_folds}")
            print(f"[INFO] Классы: {class_labels}")
        
        # Загрузка данных
        X, y, categorical, feature_names = dataset.get_data(target=dataset.default_target_attribute)
        
        if verbose:
            print(f"[INFO] Данные загружены. Размер X: {X.shape}, размер y: {y.shape}")
        
        # Проверки данных
        if X is None or (isinstance(X, pd.DataFrame) and X.empty) or (isinstance(X, np.ndarray) and X.size == 0) or len(X) == 0:
            raise ValueError("X пустой или None")
        if y is None or (isinstance(y, pd.Series) and y.empty) or (isinstance(y, np.ndarray) and len(y) == 0) or len(y) == 0:
            raise ValueError("y пустой или None")
        if len(X) != len(y):
            raise ValueError(f"Несоответствие размеров: X={len(X)}, y={len(y)}")
        
        # Преобразование меток
        # Преобразуем y в Series если нужно
        if not isinstance(y, pd.Series):
            y = pd.Series(y)
        
        # Создаем mapping, нормализуя значения (приводим к строке для сравнения)
        label_mapping = {}
        for k, v in enumerate(class_labels):
            key = str(v).strip()
            label_mapping[key] = k
            label_mapping[key.lower()] = k
            label_mapping[key.upper()] = k
        
        # Преобразуем значения y в строки для mapping
        y_str = y.astype(str).str.strip()
        y_mapped = y_str.map(label_mapping)
        
        # Проверяем на NaN перед преобразованием
        if y_mapped.isna().any():
            missing = y_str[y_mapped.isna()].unique()
            raise ValueError(
                f"Не удалось сопоставить значения в y: {missing}. "
                f"Ожидаемые классы: {class_labels}. "
                f"Уникальные значения в y: {y.unique()}"
            )
        
        # Безопасное преобразование в numpy array
        # Проверяем, является ли y_mapped уже числовым (int или bool)
        if pd.api.types.is_integer_dtype(y_mapped) or pd.api.types.is_bool_dtype(y_mapped):
            # Если уже числовой, просто преобразуем в numpy array
            y = y_mapped.values
        else:
            # Используем pd.to_numeric для безопасного преобразования
            y = pd.to_numeric(y_mapped, errors='coerce').values
            
            # Проверяем на NaN после преобразования
            if np.isnan(y).any():
                nan_indices = np.where(np.isnan(y))[0]
                raise ValueError(
                    f"Обнаружены NaN значения в y после преобразования на индексах: {nan_indices[:10]}. "
                    f"Уникальные значения в y_mapped: {y_mapped.unique()}"
                )
        
        # Преобразуем в int32
        y = y.astype(np.int32)
        
        # Проверяем на наличие некорректных значений
        if np.any(y < 0) or np.any(y >= len(class_labels)):
            raise ValueError(f"Обнаружены некорректные значения в y после преобразования: {np.unique(y)}")
            raise ValueError(f"Обнаружены некорректные значения в y после преобразования: {np.unique(y)}")
        
        
        if verbose:
            print(f"[INFO] Уникальные классы в y: {np.unique(y)}")
            print(f"[INFO] Распределение классов: {pd.Series(y).value_counts().to_dict()}")
        
        # ========== Стратифицированное разделение ==========
        if verbose:
            print("\n[INFO] ========== Стратифицированное разделение на train/test ==========")
        
        # Сначала разделяем индексы, чтобы потом использовать их для формирования датафрейма
        indices = np.arange(len(X))
        train_indices, test_indices = train_test_split(
            indices,
            test_size=test_size,
            random_state=random_state,
            stratify=y
        )
        
        # Теперь разделяем данные используя индексы
        if isinstance(X, pd.DataFrame):
            X_train = X.iloc[train_indices].reset_index(drop=True)
            X_test = X.iloc[test_indices].reset_index(drop=True)
        else:
            X_train = X[train_indices]
            X_test = X[test_indices]
        y_train = y[train_indices]
        y_test = y[test_indices]
        
        if verbose:
            print(f"[INFO] Train размер: {len(X_train)}, Test размер: {len(X_test)}")
            print(f"[INFO] Распределение классов в train: {pd.Series(y_train).value_counts().to_dict()}")
            print(f"[INFO] Распределение классов в test: {pd.Series(y_test).value_counts().to_dict()}")
        
        # Обучение на train
        automl_stratified = AutoML(
            task='classification',
            use_preprocessing_pipeline=True,
            feature_selector_type=None,
            use_val_test_pipeline=False,
            auto_models_init_kwargs={
                "metric": "roc_auc",
                "time_series": False,
                "models_list": ["all"],
                "blend": True,
                "stack": True,
                "n_splits": n_folds,
            },
            n_jobs=n_jobs,
            random_state=random_state,
        )
        
        if verbose:
            print("[INFO] Начало обучения модели на train данных...")
        
        automl_stratified = automl_stratified.fit(
            X_train, y_train,
            auto_model_fit_kwargs={"tuning_timeout": tuning_timeout}
        )
        
        if automl_stratified is None or not hasattr(automl_stratified, 'auto_model'):
            raise ValueError("AutoML (стратифицированный) не обучен корректно")
        
        # Предсказания на train и test
        predictions_train = automl_stratified.predict(X_train)
        predictions_test = automl_stratified.predict(X_test)
        
        # Обработка предсказаний
        if len(predictions_train.shape) == 2 and predictions_train.shape[1] >= 2:
            predictions_train_proba = predictions_train[:, 1]
        else:
            predictions_train_proba = predictions_train
        
        if len(predictions_test.shape) == 2 and predictions_test.shape[1] >= 2:
            predictions_test_proba = predictions_test[:, 1]
        else:
            predictions_test_proba = predictions_test
        
        # Метрики
        oof_score = automl_stratified.auto_model.best_score if hasattr(automl_stratified.auto_model, 'best_score') else None
        train_score = roc_auc_score(y_train, predictions_train_proba) if predictions_train_proba is not None else None
        test_score = roc_auc_score(y_test, predictions_test_proba) if predictions_test_proba is not None else None
        
        if verbose:
            print(f"[INFO] OOF score (train): {oof_score}")
            print(f"[INFO] ROC-AUC (train): {train_score}")
            print(f"[INFO] ROC-AUC (test): {test_score}")
        
        # ========== Формирование датафрейма с результатами ==========
        n_total = len(X)
        
        # Создаем датафрейм с правильной структурой
        df_results = pd.DataFrame(index=range(n_total))
        
        # Истинные значения для всех данных
        df_results['y_true'] = y
        df_results['is_test'] = False
        
        # Заполняем данные используя сохраненные индексы
        df_results.loc[train_indices, 'predictions_train'] = predictions_train_proba
        df_results.loc[test_indices, 'predictions_test'] = predictions_test_proba
        df_results.loc[train_indices, 'y_true_train'] = y_train
        df_results.loc[test_indices, 'y_true_test'] = y_test
        df_results.loc[test_indices, 'is_test'] = True
        
        # Добавляем метрики как отдельные колонки (одинаковые для всех строк)
        df_results['oof_score'] = oof_score
        df_results['train_score'] = train_score
        df_results['test_score'] = test_score
        df_results['task_id'] = task_id
        df_results['dataset_name'] = dataset.name
        
        if verbose:
            print(f"\n[INFO] Датафрейм создан. Размер: {df_results.shape}")
            print(f"[INFO] Колонки: {df_results.columns.tolist()}")
        
        return df_results
        
    except Exception as e:
        error_msg = f"Ошибка при выполнении пайплайна: {type(e).__name__}: {e}"
        if verbose:
            print(f"[ERROR] {error_msg}")
            print(f"[ERROR] Трассировка: {traceback.format_exc()}")
        
        # Возвращаем пустой датафрейм с правильной структурой
        return pd.DataFrame({
            'y_true': [],
            'is_test': [],
            'predictions_train': [],
            'predictions_test': [],
            'y_true_train': [],
            'y_true_test': [],
            'oof_score': [],
            'train_score': [],
            'test_score': [],
            'task_id': [],
            'dataset_name': [],
        })


## Пакетная обработка task_id из bench_merged.csv


In [3]:
def process_benchmark_csv(
    input_csv_path: str = "bench_merged.csv",
    output_csv_path: str = "bench_with_metrics.csv",
    sort_by: str = "num_cells",
    ascending: bool = True,
    test_size: float = 0.2,
    random_state: int = 42,
    n_jobs: int = 16,
    tuning_timeout: int = 10,
    verbose: bool = True
) -> pd.DataFrame:
    """
    Обрабатывает все task_id из bench_merged.csv, вычисляет метрики и сохраняет результаты.
    
    Параметры:
    ----------
    input_csv_path : str, default="bench_merged.csv"
        Путь к входному CSV файлу с task_id
    output_csv_path : str, default="bench_with_metrics.csv"
        Путь к выходному CSV файлу с результатами
    sort_by : str, default="num_cells"
        Колонка для сортировки task_id
    ascending : bool, default=True
        Сортировать по возрастанию
    test_size : float, default=0.2
        Доля данных для тестового набора
    random_state : int, default=42
        Random state для воспроизводимости
    n_jobs : int, default=16
        Количество параллельных процессов
    tuning_timeout : int, default=10
        Таймаут для тюнинга модели (в секундах)
    verbose : bool, default=True
        Выводить ли информационные сообщения
    
    Возвращает:
    -----------
    pd.DataFrame
        Датафрейм с результатами обработки всех task_id
    """
    import traceback
    from pathlib import Path
    
    # Метрики, которые должны быть заполнены
    metric_columns = ['oof_score', 'train_score', 'test_score']
    
    # Шаг 1: Чтение и подготовка данных
    if verbose:
        print(f"[INFO] Чтение файла {input_csv_path}...")
    
    if not Path(input_csv_path).exists():
        raise FileNotFoundError(f"Файл {input_csv_path} не найден")
    
    bench_df = pd.read_csv(input_csv_path)
    
    if 'task_id' not in bench_df.columns:
        raise ValueError(f"В файле {input_csv_path} отсутствует колонка 'task_id'")
    
    if verbose:
        print(f"[INFO] Загружено {len(bench_df)} строк из {input_csv_path}")
    
    # Извлечение и сортировка task_id
    if sort_by in bench_df.columns:
        bench_df = bench_df.sort_values(by=sort_by, ascending=ascending)
        if verbose:
            print(f"[INFO] Данные отсортированы по колонке '{sort_by}' ({'по возрастанию' if ascending else 'по убыванию'})")
    else:
        if verbose:
            print(f"[WARNING] Колонка '{sort_by}' не найдена, сортировка не выполнена")
    
    task_ids = bench_df['task_id'].unique().tolist()
    
    if verbose:
        print(f"[INFO] Найдено {len(task_ids)} уникальных task_id")
    
    # Шаг 4: Загрузка существующих результатов (если есть)
    if Path(output_csv_path).exists():
        if verbose:
            print(f"[INFO] Загрузка существующих результатов из {output_csv_path}...")
        results_df = pd.read_csv(output_csv_path)
        
        # Проверяем, что все исходные колонки присутствуют
        missing_cols = set(bench_df.columns) - set(results_df.columns)
        if missing_cols:
            if verbose:
                print(f"[WARNING] В результатах отсутствуют колонки: {missing_cols}. Они будут добавлены.")
            # Добавляем недостающие колонки
            for col in missing_cols:
                results_df[col] = None
    else:
        if verbose:
            print(f"[INFO] Файл {output_csv_path} не существует, будет создан новый")
        results_df = pd.DataFrame()
    
    # Убеждаемся, что колонки с метриками присутствуют
    for col in metric_columns:
        if col not in results_df.columns:
            results_df[col] = None
    
    # Шаг 2: Обработка каждого task_id
    processed_count = 0
    skipped_count = 0
    error_count = 0
    
    for idx, task_id in enumerate(task_ids, 1):
        if verbose:
            print(f"\n{'='*80}")
            print(f"[INFO] Обработка task_id {task_id} ({idx}/{len(task_ids)})")
            print(f"{'='*80}")
        
        # Проверка, нужно ли обрабатывать этот task_id
        needs_processing = False
        
        if len(results_df) == 0:
            # Если результатов нет, нужно обработать
            needs_processing = True
        else:
            # Ищем строку с таким task_id
            existing_row = results_df[results_df['task_id'] == task_id]
            
            if len(existing_row) == 0:
                # Строки нет - нужно обработать
                needs_processing = True
                if verbose:
                    print(f"[INFO] task_id {task_id} не найден в результатах, требуется обработка")
            else:
                # Строка есть - проверяем метрики
                row_metrics = existing_row[metric_columns].iloc[0]
                
                # Проверяем, заполнены ли все метрики (не NaN и не None)
                metrics_filled = all(
                    pd.notna(row_metrics[col]) and row_metrics[col] is not None 
                    for col in metric_columns
                )
                
                if not metrics_filled:
                    needs_processing = True
                    if verbose:
                        print(f"[INFO] task_id {task_id} найден, но метрики не заполнены, требуется обработка")
                else:
                    if verbose:
                        print(f"[INFO] task_id {task_id} уже обработан, пропускаем")
                    skipped_count += 1
                    continue
        
        if needs_processing:
            try:
                # Запускаем расчёт метрик
                predictions_df = run_automl_pipeline(
                    task_id=task_id,
                    test_size=test_size,
                    random_state=random_state,
                    n_jobs=n_jobs,
                    tuning_timeout=tuning_timeout,
                    verbose=verbose
                )
                
                if len(predictions_df) == 0:
                    if verbose:
                        print(f"[WARNING] Не удалось получить результаты для task_id {task_id}")
                    error_count += 1
                    continue
                
                # Извлекаем метрики (они одинаковые для всех строк)
                metrics = {
                    'oof_score': predictions_df['oof_score'].iloc[0] if 'oof_score' in predictions_df.columns else None,
                    'train_score': predictions_df['train_score'].iloc[0] if 'train_score' in predictions_df.columns else None,
                    'test_score': predictions_df['test_score'].iloc[0] if 'test_score' in predictions_df.columns else None,
                }
                
                # Находим исходную строку для этого task_id
                original_row = bench_df[bench_df['task_id'] == task_id].iloc[0].copy()
                
                # Добавляем метрики к исходной строке
                for metric_name, metric_value in metrics.items():
                    original_row[metric_name] = metric_value
                
                # Обновляем или добавляем строку в результаты
                if len(results_df) > 0 and task_id in results_df['task_id'].values:
                    # Обновляем существующую строку
                    mask = results_df['task_id'] == task_id
                    results_df.loc[mask, metric_columns] = [
                        metrics.get('oof_score'),
                        metrics.get('train_score'),
                        metrics.get('test_score')
                    ]
                    # Также обновляем другие колонки из исходного файла
                    for col in bench_df.columns:
                        if col in original_row and col in results_df.columns:
                            results_df.loc[mask, col] = original_row[col]
                else:
                    # Добавляем новую строку
                    # Если results_df пустой, создаём его с правильными колонками
                    if len(results_df) == 0:
                        # Создаём DataFrame с колонками из bench_df + метрики
                        all_cols = list(bench_df.columns) + metric_columns
                        results_df = pd.DataFrame(columns=all_cols)
                    # Убеждаемся, что все колонки присутствуют
                    new_row = original_row.copy()
                    # Добавляем недостающие колонки из results_df
                    for col in results_df.columns:
                        if col not in new_row.index:
                            new_row[col] = None
                    # Преобразуем в DataFrame с правильным порядком колонок
                    new_row_df = pd.DataFrame([new_row], columns=results_df.columns)
                    results_df = pd.concat([results_df, new_row_df], ignore_index=True)
                
                # Шаг 5: Сохранение после каждого расчёта
                results_df.to_csv(output_csv_path, index=False)
                
                if verbose:
                    print(f"[INFO] Результаты для task_id {task_id} сохранены в {output_csv_path}")
                    oof_str = f"{metrics['oof_score']:.4f}" if metrics['oof_score'] is not None else "None"
                    train_str = f"{metrics['train_score']:.4f}" if metrics['train_score'] is not None else "None"
                    test_str = f"{metrics['test_score']:.4f}" if metrics['test_score'] is not None else "None"
                    print(f"[INFO] Метрики: OOF={oof_str}, Train={train_str}, Test={test_str}")
                
                processed_count += 1
                
            except Exception as e:
                error_msg = f"Ошибка при обработке task_id {task_id}: {type(e).__name__}: {e}"
                if verbose:
                    print(f"[ERROR] {error_msg}")
                    print(f"[ERROR] Трассировка: {traceback.format_exc()}")
                
                # Сохраняем информацию об ошибке в результатах
                if len(results_df) == 0 or task_id not in results_df['task_id'].values:
                    # Добавляем строку с ошибкой
                    original_row = bench_df[bench_df['task_id'] == task_id].iloc[0].copy()
                    for metric_name in metric_columns:
                        original_row[metric_name] = None
                    original_row['error'] = str(e)
                    # Если results_df пустой, создаём его с правильными колонками
                    if len(results_df) == 0:
                        # Создаём DataFrame с колонками из bench_df + метрики + error
                        all_cols = list(bench_df.columns) + metric_columns + ['error']
                        results_df = pd.DataFrame(columns=all_cols)
                    # Убеждаемся, что все колонки присутствуют
                    for col in results_df.columns:
                        if col not in original_row.index:
                            original_row[col] = None
                    # Преобразуем в DataFrame с правильным порядком колонок
                    new_row_df = pd.DataFrame([original_row], columns=results_df.columns)
                    results_df = pd.concat([results_df, new_row_df], ignore_index=True)
                else:
                    # Обновляем строку с ошибкой
                    if 'error' not in results_df.columns:
                        results_df['error'] = None
                    mask = results_df['task_id'] == task_id
                    results_df.loc[mask, 'error'] = str(e)
                    # Обнуляем метрики для строки с ошибкой
                    results_df.loc[mask, metric_columns] = [None, None, None]
                
                # Сохраняем даже при ошибке
                results_df.to_csv(output_csv_path, index=False)
                
                error_count += 1
    
    # Итоговая статистика
    if verbose:
        print(f"\n{'='*80}")
        print(f"[INFO] Обработка завершена!")
        print(f"[INFO] Обработано: {processed_count}")
        print(f"[INFO] Пропущено (уже обработано): {skipped_count}")
        print(f"[INFO] Ошибок: {error_count}")
        print(f"[INFO] Всего task_id: {len(task_ids)}")
        print(f"[INFO] Результаты сохранены в {output_csv_path}")
        print(f"{'='*80}")
    
    return results_df


In [ ]:
# Пример использования функции для пакетной обработки
# Раскомментируйте и запустите для обработки всех task_id из bench_merged.csv

results_bench = process_benchmark_csv(
    input_csv_path="bench_merged.csv",
    output_csv_path="bench_with_metrics.csv",
    sort_by="num_cells",
    ascending=True,
    test_size=0.2,
    random_state=42,
    n_jobs=16,
    tuning_timeout=60,
    verbose=True
)

# Просмотр результатов
print("\n[INFO] Первые строки результатов:")
print(results_bench.head())

print("\n[INFO] Статистика по метрикам:")
print(results_bench[['task_id', 'oof_score', 'train_score', 'test_score']].describe())


[INFO] Чтение файла bench_merged.csv...
[INFO] Загружено 41 строк из bench_merged.csv
[INFO] Данные отсортированы по колонке 'num_cells' (по возрастанию)
[INFO] Найдено 41 уникальных task_id
[INFO] Загрузка существующих результатов из bench_with_metrics.csv...

[INFO] Обработка task_id 359955 (1/41)
[INFO] task_id 359955 уже обработан, пропускаем

[INFO] Обработка task_id 146818 (2/41)
[INFO] task_id 146818 уже обработан, пропускаем

[INFO] Обработка task_id 168757 (3/41)
[INFO] task_id 168757 не найден в результатах, требуется обработка
[INFO] Начало загрузки данных из OpenML (task_id=168757)...
[INFO] Задача загружена: 168757
[INFO] Датасет загружен: credit-g
[INFO] Количество фолдов: 9
[INFO] Классы: ['bad', 'good']
[INFO] Данные загружены. Размер X: (1000, 20), размер y: (1000,)
[INFO] Уникальные классы в y: [0 1]
[INFO] Распределение классов: {1: 700, 0: 300}

[INFO] ========== Стратифицированное разделение на train/test ==========
[INFO] Train размер: 800, Test размер: 200
[INFO]

In [ ]:
task_id,task_name,lightautoml,MLJAR_B,MLJAR_P,TPOT,constantpredictor,RandomForest,TunedRandomForest_star,dataset_size,num_features,num_cells,oof_score,train_score,test_score
359955,blood-tr...,0.753(0.058),-,0.753(0.053),0.724(0.102),0.500(0.000),0.686(0.064),0.686(0.065),748,5,3740,0.7488340128490241,0.7504092537682232,0.7982456140350878
146820,wilt,0.994(0.007),0.999(0.000)8,0.995(0.009),0.996(0.004),0.500(0.000),0.989(0.012),0.991(0.010),4839,6,29034,0.8272220320425212,0.9429077634257431,0.8402544507893853
168350,phoneme,0.966(0.008),-,0.967(0.008),0.969(0.008),0.500(0.000),0.965(0.009),0.966(0.009),5404,6,32424,0.9577249643016199,0.9998327968900221,0.9422184418716039
359956,qsar-bio...,0.934(0.034),0.932(nan)9,0.937(0.031),0.926(0.045),0.500(0.000),0.932(0.033),0.935(0.029),1055,42,44310,0.9351347958447102,0.9805228635093997,0.94738430583501
359968,churn,0.926(0.020),0.928(0.021),0.925(0.019),0.919(0.025),0.500(0.000),0.916(0.023),0.912(0.021),5000,21,105000,0.9319818855716376,0.9991613690572966,0.935534474359927
359972,sylvine,0.988(0.003),0.992(0.003),0.992(0.003),0.992(0.002),0.500(0.000),0.983(0.004),0.984(0.004),5124,21,107604,0.9840855146472401,0.9990515301931936,0.9888561464424952
190137,ozone-le...,0.930(0.018),-,0.929(0.019),0.928(0.026),0.500(0.000),0.920(0.036),0.921(0.033),2534,73,184982,0.9113801671932596,0.9348341232227488,0.9129605263157895
359975,Satellite,0.986(0.025),-,0.993(0.007),0.984(0.038),0.500(0.000),0.985(0.029),0.984(0.031),5100,37,188700,0.98106135986733,0.9884162520729685,0.9697512437810946
190411,ada,0.921(0.018),0.921(0.018),0.921(0.018),0.917(0.019),0.500(0.000),0.903(0.014),0.902(0.016),4147,49,203203,0.9155484706430305,0.9200784190684618,0.9045229648991785
168911,jasmine,0.880(0.018),0.885(0.016),0.886(0.019),0.886(0.013),0.500(0.000),0.888(0.016),0.888(0.017),2984,145,432680,0.8706265330564531,0.9635618719470502,0.8752328791721847
